# Lab 5

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fayam1/Geospatial-Data-Management-course/blob/main/lab_05.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

## Question 1

Visualize the [USGS Watershed Boundary Dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_WBD_2017_HUC04) with outline color only, no fill color.

In [2]:
# Add your code here.
import geemap
import ee
geemap.ee_initialize()
# Load the USGS watershed  
fc=ee.FeatureCollection("USGS/WBD/2017/HUC04")
# Define the styling for the outline (no fill)
style = {
    'color': 'blue',  # Outline color
    'fillColor': '00000000',  # Transparent fill color
    'width': 2  # Outline width
}
map=geemap.Map(center=[40,-100],zoom=4)
#Add text and layers
text = "Made by Faissal Bozi"
map.add_text(text, fontsize=20, position='bottomright')
map.addLayer(fc.style(**style),{},'Watershed Boundary')
map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Question 2 

Filter the USGS Watershed Boundary dataset and select the watershed that intersects the county of your choice.

In [ ]:
# Add your code here.
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter counties by name "Knox" and STATEFP=18
Knox= counties.filter(ee.Filter.And
                                (ee.Filter.eq('NAME','Knox'),
                                 ee.Filter.eq('STATEFP','18')
                                ))
# Filter the watershed boundary that intersects Knox county boundary and merge the results using dissolve()
Knoxwb = (ee.FeatureCollection('USGS/WBD/2017/HUC06')\
                          .filter(ee.Filter\
                                  .intersects('.geo',Knox.geometry()))\
                                  .geometry().dissolve())
                           
                                 
map1=geemap.Map()
#Add text and layers
text = "Made by Faissal Bozi"
map1.add_text(text, fontsize=20, position='bottomright')
map1.addLayer(fc.style(**style),{},'Watershed Boundary')
map1.addLayer(Knoxwb,{'color':'lightblue'},'Knox County Watersheds')
map1.addLayer(Knox,{'color':'red'},'Knox County')
map1.centerObject(Knoxwb)
map1

## Question 3

Clip the [USGS 3DEP 10m DEM](https://developers.google.com/earth-engine/datasets/catalog/USGS_3DEP_10m) with the watershed that intersects the county of your choice. Display the DEM with a proper color palette and color bar.

In [ ]:
# Add your code here.
import geemap
import ee

# Initialize Earth Engine
ee.Initialize()

# Load the USGS 3DEP 10m DEM
dem = ee.Image('USGS/3DEP/10m')

# Load the US Census Counties dataset
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter for Knox County, Indiana
knox_county = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Knox'),
    ee.Filter.eq('STATEFP', '18')
))

# Find the watershed that intersects with Knox County
Knox_watershed =( ee.FeatureCollection('USGS/WBD/2017/HUC06') \
    .filter(ee.Filter.intersects('.geo', knox_county.geometry()))).geometry().dissolve()

# Clip the DEM with the intersecting watershed
clipped_dem = dem.clip(intersecting_watershed)

# Define visualization parameters
vis_params = {
    'min': 0,
    'max': 400,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}

# Create a map centered on the clipped DEM
map2 = geemap.Map()

# Add the clipped DEM to the map and Knox_watershed

map2.addLayer(Knox_watershed, {'color': 'red', 'fillOpacity': 0.0, 'fillColor': '#00000000', 'width': 2}, 'Knox watershed')

map2.addLayer(clipped_dem, vis_params, 'Clipped DEM')

text = "Made by Faissal Bozi"
map2.add_text(text, fontsize=20, position='bottomright')
# Add a color bar legend
map2.add_colorbar(vis_params,orientation="vertical", label="Elevation (m)",font_size=8)
map2.centerObject(Knox_watershed)

# Display the map
map2


## Question 4

Use the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) and [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) to create a split-panel map for visualizing land cover change (2001-2019) for a US state of your choice. Make sure you add the NLCD legend to the map.

In [ ]:
# Add your code here.
# Load the NLCD datasets for 2001 and 2019 and select the land cover band
ncld2001=ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2001').select('landcover')
ncld2019=ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019').select('landcover')
# Load the US Census States dataset
states=ee.FeatureCollection("TIGER/2018/States")
california_state=states.filter(ee.Filter.eq('NAME','California'))

# Clip the NLCD datasets to the California boundary
ncld2001_clip = ncld2001.clip(california_state.geometry())
ncld2019_clip =ncld2019.clip(california_state.geometry())

map3=geemap.Map()
# Create tile layers for the split map
left_layer=geemap.ee_tile_layer(ncld2001_clip,{},'2001 Land Cover')
right_layer=geemap.ee_tile_layer(ncld2019_clip,{},'2019 Land Cover')                                
 # Add the split map layers                               
map3.split_map(left_layer,right_layer,add_close_button=True)
# Add the California state boundary to both panels
map3.addLayer(california_state.style(color='red', fillColor='00000000', width=2),{},'California State')
map3.centerObject(california_state)
#Add text
text = "Made by Faissal Bozi"
map3.add_text(text, fontsize=20, position='bottomleft')
# Add the NLCD legend
map3.add_legend(builtin_legend='NLCD',position='bottomright')
map3


## Questions 5

Download OpenStreetMap data for a city of your choice and visualize the city boundary and restaurants in the city.

In [9]:
import geemap
import geemap.osm as osm

# Retrieve geographical data for Knoxville
gdf_knoxville = geemap.osm_to_gdf("Knoxville, Tennessee")
fc = geemap.osm_to_ee("Knoxville, Tennessee")
# Define a bounding box around Knoxville to ensure all data within this area is captured
minx, miny, maxx, maxy = gdf_knoxville.total_bounds  # [minx, miny, maxx, maxy]
# Retrieve restaurant data within the bounding box using geemap
gdf_restaurants = osm.osm_gdf_from_bbox(
    north=maxy, south=miny, east=maxx, west=minx, tags={"amenity": "restaurant"}
)

# Create a map centered on Knoxville
map4 = geemap.Map(center=[35.9606, -83.9207], zoom=9)

# Add geographical data to the map with a layer name 'Knoxville'
#map4.add_gdf(gdf_knoxville, 'Knoxville')
map4.addLayer(fc,{},'Knoxville')
text = "Made by Faissal Bozi"
map4.add_text(text, fontsize=20, position='bottomright')
# Add restaurant data to the map with a layer name 'Restaurants'
map4.add_gdf(gdf_restaurants, 'Restaurants')

# Display the map
map4


Map(center=[35.9606, -83.9207], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…